In [1]:
import os
import cv2
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler


In [2]:
# Загрузка изображений из директории
def load_images_from_directory(directory, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                # Масштабируем изображение к нужному размеру
                img_resized = cv2.resize(img, target_size)
                img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)  # Преобразуем в градации серого
                images.append(img_gray.flatten())  # Преобразуем изображение в одномерный массив
    images = np.array(images)
    return images

In [3]:
# Обучение One-Class SVM на нормальных данных
def train_one_class_svm(normal_data):
    scaler = StandardScaler()
    normal_data_scaled = scaler.fit_transform(normal_data)

    # Обучение модели One-Class SVM
    model = svm.OneClassSVM(gamma='auto', kernel='rbf', nu=0.1)  # nu — это параметр для управления уровнем аномалий
    model.fit(normal_data_scaled)

    return model, scaler

In [4]:
# Проверка изображения на аномалии
def is_anomaly(image, model, scaler):
    img_resized = cv2.resize(image, (128, 128))
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    img_flattened = img_gray.flatten().reshape(1, -1)  # Преобразуем изображение в одномерный массив
    img_scaled = scaler.transform(img_flattened)

    prediction = model.predict(img_scaled)
    return prediction == -1  # -1 означает аномалию


In [6]:
# Основной код
if __name__ == "__main__":
    # Путь к данным
    data_dir = "object3_cut"

    # Загрузка нормальных изображений
    normal_images = load_images_from_directory(data_dir)

    # Обучение модели One-Class SVM
    model, scaler = train_one_class_svm(normal_images)

    # Проверка новых изображений на наличие аномалий
    source_dir = "object3_new"
    for filename in os.listdir(source_dir):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img_path = os.path.join(source_dir, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue

            if is_anomaly(img, model, scaler):
                print(f"Аномалия обнаружена на изображении: {img_path}")
            else:
                print(f"Все в порядке на изображении: {img_path}")

Аномалия обнаружена на изображении: object3_new\21.08.2024_10.27.20.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.28.17.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.29.14.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.31.09.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.35.54.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.36.52.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.37.49.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.38.46.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.39.43.png
Аномалия обнаружена на изображении: object3_new\21.08.2024_10.45.28.png
